In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import requests
import zipfile


In [4]:
data_path = "../../../data/book_crossing/"

In [5]:
# Load the data
books = pd.read_csv(data_path+'BX-Books.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
users = pd.read_csv(data_path+'BX-Users.csv', sep=';', error_bad_lines=False, warn_bad_lines=False,encoding="latin-1")
ratings = pd.read_csv(data_path+'BX-Book-Ratings.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")

/var/folders/jv/v8v_byq94r70bn81gkfc9m6m0000gn/T/ipykernel_21478/170911166.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(data_path+'BX-Books.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
/var/folders/jv/v8v_byq94r70bn81gkfc9m6m0000gn/T/ipykernel_21478/170911166.py:2: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(data_path+'BX-Books.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
/var/folders/jv/v8v_byq94r70bn81gkfc9m6m0000gn/T/ipykernel_21478/170911166.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(data_path+'BX-Books.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
/var/folders/j

In [6]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [8]:
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [9]:
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [11]:
ratings['Book-Rating'].unique()

array([ 0,  5,  3,  6,  8,  7, 10,  9,  4,  1,  2])

In [13]:
# Preprocess the data
ratings = ratings.merge(books[['ISBN', 'Book-Title']], on='ISBN')

In [14]:
ratings

,User-ID,ISBN,Book-Rating,Book-Title
0,276725,034545104X,0,Flesh Tones: A Novel
1,2313,034545104X,5,Flesh Tones: A Novel
2,6543,034545104X,0,Flesh Tones: A Novel
3,8680,034545104X,5,Flesh Tones: A Novel
4,10314,034545104X,9,Flesh Tones: A Novel
...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless
1031132,276688,1575660792,7,Gray Matter
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries)


In [15]:
ratings = ratings.drop(['ISBN'], axis=1)

In [16]:
ratings

,User-ID,Book-Rating,Book-Title
0,276725,0,Flesh Tones: A Novel
1,2313,5,Flesh Tones: A Novel
2,6543,0,Flesh Tones: A Novel
3,8680,5,Flesh Tones: A Novel
4,10314,9,Flesh Tones: A Novel
...,...,...,...
1031131,276688,0,Mostly Harmless
1031132,276688,7,Gray Matter
1031133,276690,0,Triplet Trouble and the Class Trip (Triplet Tr...
1031134,276704,0,A Desert of Pure Feeling (Vintage Contemporaries)


In [17]:
# Filter books with at least min_book_ratings ratings
min_book_ratings = 10
book_rating_counts = ratings['Book-Title'].value_counts()

In [18]:
book_rating_counts

Wild Animus                                                                   2502
The Lovely Bones: A Novel                                                     1295
The Da Vinci Code                                                              898
A Painted House                                                                838
The Nanny Diaries: A Novel                                                     828
                                                                              ... 
Asphyxie                                                                         1
Marathon : The Ultimate Training Guide                                           1
Pearl Jam and Eddie Vedder: None Too Fragile                                     1
Milk It!: Collected Musings on the Alternative Music Explosion of the '90s       1
Perplexing Lateral Thinking Puzzles: Scholastic Edition                          1
Name: Book-Title, Length: 241071, dtype: int64

In [19]:
filtered_books = book_rating_counts[book_rating_counts >= min_book_ratings].index

In [20]:
filtered_books

Index(['Wild Animus', 'The Lovely Bones: A Novel', 'The Da Vinci Code',
       'A Painted House', 'The Nanny Diaries: A Novel',
       'Bridget Jones's Diary', 'The Secret Life of Bees',
       'Divine Secrets of the Ya-Ya Sisterhood: A Novel',
       'The Red Tent (Bestselling Backlist)', 'Angels &amp; Demons',
       ...
       'Compulsion', 'The Spectator Bird (Contemporary American Fiction)',
       'The Deadliest Game (Tom Clancy's Net Force; Young Adult)',
       'More Tales to Give You Goosebumps: Ten Spooky Stories (Goosebumps Special Edition, No 2)',
       'A Man Rides Through (Mordant's Need, Vol 2)',
       'When the Elephants Dance',
       'Of Mice and Men/Cannery Row (2 Books in 1)',
       'What to Say When You Talk to Your Self', 'The Invisible Ring',
       'Chicken Soup for the Christian Soul (Chicken Soup for the Soul)'],
      dtype='object', length=17447)

In [21]:
ratings = ratings[ratings['Book-Title'].isin(filtered_books)]

In [22]:
ratings

,User-ID,Book-Rating,Book-Title
0,276725,0,Flesh Tones: A Novel
1,2313,5,Flesh Tones: A Novel
2,6543,0,Flesh Tones: A Novel
3,8680,5,Flesh Tones: A Novel
4,10314,9,Flesh Tones: A Novel
...,...,...,...
1031125,276688,0,Coyote Waits
1031126,276688,0,Sacred Clowns
1031128,276688,0,Mrs. Pollifax and the Second Thief
1031131,276688,0,Mostly Harmless


In [23]:
ratings.count()

User-ID        572609
Book-Rating    572609
Book-Title     572609
dtype: int64

In [25]:
# Filter users with at least min_user_ratings ratings
min_user_ratings = 5
user_rating_counts = ratings['User-ID'].value_counts()

In [26]:
user_rating_counts

11676     5827
35859     2741
153662    2409
76352     2132
198711    1837
          ... 
113200       1
125376       1
234534       1
362          1
276556       1
Name: User-ID, Length: 68518, dtype: int64

In [27]:
filtered_users = user_rating_counts[user_rating_counts >= min_user_ratings].index

In [28]:
filtered_users

Int64Index([ 11676,  35859, 153662,  76352, 198711,  16795, 204864,  98391,
            102967,  55492,
            ...
              5416, 128821, 227837,  60419,   3079, 276681, 104141, 130939,
            131057, 274418],
           dtype='int64', length=14322)

In [29]:
ratings = ratings[ratings['User-ID'].isin(filtered_users)]

In [30]:
ratings

,User-ID,Book-Rating,Book-Title
1,2313,5,Flesh Tones: A Novel
2,6543,0,Flesh Tones: A Novel
3,8680,5,Flesh Tones: A Novel
4,10314,9,Flesh Tones: A Novel
5,23768,0,Flesh Tones: A Novel
...,...,...,...
1031125,276688,0,Coyote Waits
1031126,276688,0,Sacred Clowns
1031128,276688,0,Mrs. Pollifax and the Second Thief
1031131,276688,0,Mostly Harmless


In [31]:
ratings.count()

User-ID        491897
Book-Rating    491897
Book-Title     491897
dtype: int64

In [35]:
# Create an user-item matrix
user_item_matrix = ratings.pivot_table(index='User-ID', columns='Book-Title', values='Book-Rating').fillna(0)

In [36]:
user_item_matrix

Book-Title,"Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth","Good Wives: Image and Reality in the Lives of Women in Northern New England, 1650-1750",Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)),"Q-Space (Star Trek The Next Generation, Book 47)","Q-Zone (Star Trek The Next Generation, Book 48)",!Yo!,'Salem's Lot,...AND THE HORSE HE RODE IN ON : THE PEOPLE V. KENNETH STARR,01-01-00: A Novel of the Millennium,01-01-00: The Novel of the Millennium,...,"\O\"" Is for Outlaw""","\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""","\The Happy Prince\"" and Other Stories (Penguin Popular Classics)""","\What Do You Care What Other People Think?\"": Further Adventures of a Curious Character""",e,iI Paradiso Degli Orchi,murder@maggody.com : An Arly Hanks Mystery (Arly Hanks Mysteries (Paperback)),one hundred years of solitude,stardust,why I'm like this : True Stories
User-ID,,,,,,,,,,,,,,,,,,,,,
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Compute the cosine similarity matrix
# cosine_similarities(user_item_matrix) = cosine_similarity(user_item_matrix, user_item_matrix)
cosine_sim_matrix = cosine_similarity(user_item_matrix)

In [41]:
print(cosine_sim_matrix)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [51]:
cosine_sim_matrix.shape

(14322, 14322)

In [52]:
# col
cosine_sim_matrix.mean(axis=0)

array([0.00019009, 0.00032347, 0.00117289, ..., 0.00555277, 0.000628  ,
       0.00123018])

In [53]:
# row
cosine_sim_matrix.mean(axis=1)

array([0.00019009, 0.00032347, 0.00117289, ..., 0.00555277, 0.000628  ,
       0.00123018])

In [45]:
# Split user indices into train and test set
user_indices = np.arange(user_item_matrix.shape[0])

In [46]:
user_indices

array([    0,     1,     2, ..., 14319, 14320, 14321])

In [48]:
train_user_indices, test_user_indices = train_test_split(user_indices, test_size=0.2, random_state=42)

In [49]:
test_user_indices

array([2996,  304, 7588, ...,  408, 6908, 1608])

In [50]:
test_user_indices

array([2996,  304, 7588, ...,  408, 6908, 1608])

In [55]:
# user_index를 input으로 받고 총 k개를 추천을 한다
def collaborative_filtering_recommendation(user_index, k=10):
    # user_index에 해당하는 유저의 코사인 유사도를 가져온다
    user_similarities = cosine_sim_matrix[user_index]
    # 유사도가 높은 순으로 정렬하고, 유사도가 높은 k개의 유저를 가져온다
    # np.argsort: 오름차순으로 정렬된 인덱스를 반환한다
    # [-k-1:-1]: 뒤에서 k+1번째부터 뒤에서 1번째까지
    # [::-1]: 역순으로 정렬한다
    top_k_similar_users = np.argsort(user_similarities)[-k-1:-1][::-1]
    # top_k_similar_users는 인덱스로 이에 해당하는 user_item_matrix를 가져온다
    similar_users_ratings = user_item_matrix.iloc[top_k_similar_users]
    # 평균을 구한다
    mean_ratings = similar_users_ratings.mean(axis=0)
    # 평균이 높은 순으로 정렬하고, 상위 k개의 책을 가져온다
    top_k_books = mean_ratings.sort_values(ascending=False).head(k).index
    return top_k_books

In [56]:
# random recommendation(ratings, n=10)
def random_recommendation(ratings, n=10):
    unique_books = ratings['Book-Title'].unique()
    random_books = np.random.choice(unique_books, size=n, replace=False)
    return random_books

In [57]:
random_books = random_recommendation(ratings, n=10)

In [58]:
random_books

array(['Catering to Nobody', "A Reporter's Life", 'The Elusive Flame',
       'Leaving Cold Sassy: The Unfinished Sequel to Cold Sassy Tree',
       'The Immaculate Deception', 'Indiscreet', 'A Blessing on the Moon',
       'Byzantium (Harper Fiction)', 'SEAT OF THE SOUL',
       "Damia's Children"], dtype=object)

In [59]:
for i, book in enumerate(random_books, 1):
    print(f"{i}: {book}")

1: Catering to Nobody
2: A Reporter's Life
3: The Elusive Flame
4: Leaving Cold Sassy: The Unfinished Sequel to Cold Sassy Tree
5: The Immaculate Deception
6: Indiscreet
7: A Blessing on the Moon
8: Byzantium (Harper Fiction)
9: SEAT OF THE SOUL
10: Damia's Children


In [60]:
# popularity_recommendation
def popularity_recommendation(ratings, n=10):
    popular_books = ratings.groupby('Book-Title')['Book-Rating'].count().sort_values(ascending=False).index[:n]
    return popular_books

In [61]:
popular_books = popularity_recommendation(ratings, n=10)

In [62]:
popular_books

Index(['Wild Animus', 'The Lovely Bones: A Novel', 'The Da Vinci Code',
       'The Nanny Diaries: A Novel', 'Bridget Jones's Diary',
       'A Painted House', 'The Secret Life of Bees',
       'Divine Secrets of the Ya-Ya Sisterhood: A Novel',
       'Angels &amp; Demons', 'Life of Pi'],
      dtype='object', name='Book-Title')

In [63]:
for i, book in enumerate(popular_books, 1):
    print(f"{i}: {book}")

1: Wild Animus
2: The Lovely Bones: A Novel
3: The Da Vinci Code
4: The Nanny Diaries: A Novel
5: Bridget Jones's Diary
6: A Painted House
7: The Secret Life of Bees
8: Divine Secrets of the Ya-Ya Sisterhood: A Novel
9: Angels &amp; Demons
10: Life of Pi


In [64]:
def recommend_books(user_index, strategy, k=10):
    if strategy == 'collaborative':
        tok_k_books = collaborative_filtering_recommendation(user_index, k)
    elif strategy == 'random':
        tok_k_books = random_recommendation(ratings, k)
    elif strategy == 'popularity':
        tok_k_books = popularity_recommendation(ratings, k)
    else:
        raise Exception('Please choose one of collaborative, random, popularity')
    return tok_k_books

In [65]:
def evaluate_model(strategy, k=10):
    true_positive = 0
    false_positive = 0
    false_negative = 0
    
    for user_index in test_user_indices:
        # 실제로 user가 평가한 책들
        true_books = set(user_item_matrix.iloc[user_index][user_item_matrix.iloc[user_index] > 0].index)
        # 추천한 책들
        recommended_books = set(recommend_books(user_index, strategy, k))
        
        tp = len(true_books.intersection(recommended_books))
        fp = len(recommended_books - true_books)
        fn = len(true_books - recommended_books)
        
        true_positive += tp
        false_positive += fp
        false_negative += fn
    
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    
    return precision, recall

In [66]:
# Evaluate the collaborative filtering model
cf_precision, cf_recall = evaluate_model(strategy='collaborative')

In [67]:
print(f'Collaborative filtering precision= {cf_precision:.4f}, Recall={cf_recall:.4f}')

Collaborative filtering precision= 0.3110, Recall=0.2601


In [69]:
# Evaluate the random recommendation model
random_precision, random_recall = evaluate_model(strategy='random')

In [70]:
print(f'Random recommendation precision= {random_precision:.4f}, Recall={random_recall:.4f}')

Random recommendation precision= 0.0006, Recall=0.0005


In [71]:
# Evaluate the popularity recommendation model
popularity_precision, popularity_recall = evaluate_model(strategy='popularity')

In [72]:
print(f'Popularity recommendation precision= {popularity_precision:.4f}, Recall={popularity_recall:.4f}')

Popularity recommendation precision= 0.0200, Recall=0.0167
